# Problème 1 : Equilibre de dissociation du CO2 à T et P constants

#### numéro d'équipe : 2
#### Nom, prénom et matricule des membres:
####    - Brahic Lucas
####    - Dodier Gabriel
####    - Goureau Mathieu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
from math import*

### Fonctions fournies pour la résolution du problème

In [ ]:
def calc_frac_mol(N):
    """
    Renvoie une liste contenant les fractions molaires du mélange.
    
    N: liste des quantités de matière de chaque espèce dans le mélange (même ordre que species)
    """
    return N/np.sum(N)
    

def format_X_cantera(species, X):
    """
    Renvoie une string contenant les fractions molaires du mélange formatée pour cantera.
    
    species : liste des strings des espèces dans le mélange
    N: liste des quantités de matière de chaque espèce (même ordre que species)
    """
    for i, sp in enumerate(species):
        if i==0:
            X_cant = "{}:{}".format(species[i], X[i])
        else:
            X_cant = X_cant + ", {}:{}".format(species[i], X[i])

    return X_cant

def calc_thermoprop_mix(gas, species, N, T, P):
    """
    Renvoie l'énergie interne (U), l'enthalpie (H), l'entropie (S) et l'enthalpie libre/fonction de Gibbs (G) du mélange
    avec la composition chimique N, à la température T et la pression P.
    
    gas : objet cantera représentant le mélange étudié
    species : liste des strings des espèces dans le mélange
    N : liste des quantités de matière de chaque espèce dans le mélange (même ordre que species)
    T : température du mélange
    P : pression du mélange
    """
    
    # Calcul des fraction molaires
    X = calc_frac_mol(N)
    
    # Formattage pour cantera
    X_cant = format_X_cantera(species, X)
    
    # Initialisation des paramètres du mélange
    gas.TPX = T, P, X_cant
    
    # Calcul des variables thermochimiques (penser aux unités)
    N_tot = # à remplir                     # Quantité de matière totale du mélange
    U = gas.u*1e-3*N_tot                    # Energie interne du mélange [J]
    H = gas.h*1e-3*N_tot                    # Enthalpie du mélange [J]
    S = gas.s*1e-3*N_tot                    # Entropie du mélange [J/K]
    G = gas.g*1e-3*N_tot                    # Enthalpie libre/fonction de Gibbs du mélange [J]
    
    return [U, H, S, G]

### Code principal

In [ ]:
%%time
gas=ct.Solution("mech_pb1-2.yaml")
gas.basis="molar"

# Paramètres du problème (à compléter)
T0 = 2500                                # Température initiale [K]
P = 10*ct.one_atm                                # Pression [Pa]
species = # à remplir                           # Liste des espèces dans le mélange
alpha_arr = np.arange(0, 1.00001, 0.001)        # Array des alpha

# arrays pour l'affichage
u_arr = np.zeros(alpha_arr.shape)
h_arr = np.zeros(alpha_arr.shape)
s_arr = np.zeros(alpha_arr.shape)
g_arr = np.zeros(alpha_arr.shape)
for i_a, a in enumerate(alpha_arr):
    # Composition du mélange
    N = # à remplir                                  # Liste des quantités de matière des espèces du mélange
    
    # Récupération des variables thermochimiques du mélange
    [u_arr[i_a], h_arr[i_a], s_arr[i_a], g_arr[i_a]] = calc_thermoprop_mix(gas, species, N, T0, P)

alpha_calc = # à remplir

print("A l'équilibre, la fraction de CO2 dissocié trouvé par minimisation de la fonction de Gibbs est {}".format(alpha_calc))

### Affichage des résultats

In [ ]:
# Gestion de l'affichage des résultats avec Matplotlib
fig, axs = plt.subplots(2, 2, figsize=(9,6), dpi=100)

axs[0,0].plot(alpha_arr, u_arr*1e-3)
#axs[0,0].set_title("u [J/mol de CO2 initiale]")
axs[0,0].set_ylabel("u [kJ/mol de CO2 initiale]")
axs[0,0].set_xlabel(r"quantité dissociée $\alpha$")
axs[0,0].axvline(alpha_eq, color="red", linestyle="--")

axs[0,1].plot(alpha_arr, h_arr*1e-3)
axs[0,1].set_ylabel("h [kJ/mol de CO2 initiale]")
axs[0,1].set_xlabel(r"quantité dissociée $\alpha$")
axs[0,1].axvline(alpha_eq, color="red", linestyle="--")

axs[1,0].plot(alpha_arr, g_arr*1e-3)
axs[1,0].set_ylabel("g [kJ/mol de CO2 initiale]")
axs[1,0].set_xlabel(r"quantité dissociée $\alpha$")
axs[1,0].axvline(alpha_eq, color="red", linestyle="--")

axs[1,1].plot(alpha_arr, s_arr*1e-3)
axs[1,1].set_ylabel("s [kJ/mol de CO2 initiale/K]")
axs[1,1].set_xlabel(r"quantité dissociée $\alpha$")
axs[1,1].axvline(alpha_eq, color="red", linestyle="--")

fig.suptitle("Problème 1")
fig.tight_layout()
plt.show()

### Comparaison avec la résolution direct par Cantera

In [ ]:
%%time
# Résultat avec cantera pour faire une comparaison
gas=ct.Solution("mech_pb1-2.yaml")
gas.basis="molar"

# Paramètres du problème
T0 = 2500                                           # Température initiale [K]
P = 10*ct.one_atm                                   # Pression [Pa]

# Initialisation du mélange pour cantera : T=température, P=pression, X=composition molaire du mélange
gas.TPX = T0, P, "CO2:1"

# On fait évoluer le mélange jusqu'à son état final à température et pression constante
gas.equilibrate("TP")

# Récupération de la fraction molaire de CO dans l'objet gas
alpha = gas.X[gas.species_index("CO")]                 # Fraction molaire de CO2 dissociée
n_tot = 1 + alpha/2                                    # Quantité de matière totale dans le système à l'équilibre chimique

print("A l'équilibre, la quantité de CO2 dissocié trouvée par cantera est {:.4f}".format(alpha*n_tot))